In [1]:
import  gensim
import gensim.downloader as api
import pandas as pd

from gensim.models import Word2Vec
from nltk.tokenize import RegexpTokenizer
import tqdm

In [2]:
# model = api.load("word2vec-ruscorpora-300")

In [56]:
# for n in model.most_similar(positive=[u'цедра_NOUN']):
#     print(n[0], n[1])

In [4]:
from pymystem3 import Mystem
m = Mystem()

tag_conversion = {
    'A': 'ADJ',                                                                                                                                                                                                                                                                    
    'ADV': 'ADV',                                                                                                                                                                                                                                                                    
    'ADVPRO': 'ADV',                                                                                                                                                                                                                                                                    
    'ANUM': 'ADJ',                                                                                                                                                                                                                                                                    
    'APRO': 'DET',                                                                                                                                                                                                                                                                    
    'COM': 'ADJ',                                                                                                                                                                                                                                                                    
    'CONJ': 'SCONJ',                                                                                                                                                                                                                                                                  
    'INTJ': 'INTJ',                                                                                                                                                                                                                                                                   
    'NONLEX': 'X',                                                                                                                                                                                                                                                                      
    'NUM': 'NUM',                                                                                                                                                                                                                                                                    
    'PART': 'PART',                                                                                                                                                                                                                                                                   
    'PR': 'ADP',                                                                                                                                                                                                                                                                    
    'S': 'NOUN',                                                                                                                                                                                                                                                                   
    'SPRO': 'PRON',                                                                                                                                                                                                                                                                   
    'UNKN': 'X',                                                                                                                                                                                                                                                                      
    'V':'VERB'
}

def tag(word='пожар', m=m):
    
    processed = m.analyze(word)[0]
    if not "analysis" in processed:
        raise ValueError('Wrong word')
    if not processed["analysis"]:
        raise ValueError('Wrong word')
    lemma = processed["analysis"][0]["lex"].lower().strip()
    pos = processed["analysis"][0]["gr"].split(',')[0]
    pos = pos.split('=')[0].strip()
    tagged = lemma+'_'+tag_conversion[pos]
    return tagged

In [5]:
data = pd.read_csv('data/eda_ru/eda_ru_filtered.csv')
data.head()

,id,url,title,ingredients,descr,steps,images,images_step
0,24419,https://eda.ru/recepty/vypechka-deserty/sloeni...,Слоеные булки,Вода:350 мл|Пшеничная мука:500 г|Соль:по вкусу...,NaN,1. Вотрите дрожжи в муку до образования одноро...,images/24419_1.jpg,NaN
1,24410,https://eda.ru/recepty/vypechka-deserty/italja...,Итальянское персиковое мороженое (Gelato),Молоко:6 стаканов|Сахар:1 стакан|Яичный желток...,NaN,1. В блендере измельчите очищенные от кожуры п...,images/24410_1.jpg,NaN
2,24411,https://eda.ru/recepty/vypechka-deserty/italja...,Итальянское мороженое с лимонной цедрой (Gelato),Молоко:6 стаканов|Сахар:1 стакан|Яичный желток...,NaN,1. В большой кастрюле смешайте слегка взбитые ...,images/24411_1.jpg,NaN
3,24414,https://eda.ru/recepty/salaty/zakuska-iz-kuric...,Закуска из курицы в листьях салата,Куриная грудка:1 штука|Оливковое масло:3 столо...,NaN,"1. Смешайте ингредиенты для соуса: масло, соев...",images/24414_1.jpg,NaN
4,24413,https://eda.ru/recepty/zakuski/marinovannie-ku...,Маринованные куриные ломтики,Лайм:1 штука|Чеснок:1 зубчик|Соевый соус:2 сто...,NaN,"1. Приготовьте маринад из сока и цедры лайма, ...",images/24413_1.jpg,NaN


In [6]:
titles = list(data.title.values)
titles[:5]

['Слоеные булки',
 'Итальянское персиковое мороженое (Gelato)',
 'Итальянское мороженое с лимонной цедрой (Gelato)',
 'Закуска из курицы в листьях салата',
 'Маринованные куриные ломтики']

In [7]:
steps = []
for recipe in data.steps.values:
    steps.extend(recipe.split('|'))
steps[:10]

['1. Вотрите дрожжи в муку до образования однородной крошки. Добавьте соль, потом воду. Замешивайте тесто 2 -3 минуты, пока оно не начнет собираться в ком. Оставьте на 1 час для расстойки.',
 '2. Выложите тесто на присыпанную поверхность. Слегка разровняйте его руками и раскатайте до толщины 5 мм.',
 '3. Формой вырежьте круги, слегка смочите их водой и сложите в стопки по 4 штуки.',
 '4. Разместите булки на плоском листе или перевернутом противне и оставьте на 45 минут.',
 '5. Поставьте противень с булками в разогретую до максимума духовку. Сбрызните духовку водой из пульверизатора и сразу же закройте. Уменьшите температуру до 230 градусов и пеките булки 10 – 12 минут, пока корочка не станет темно-золотистой. Охладите на решетке.',
 '1. В блендере измельчите очищенные от кожуры персики до состояния пюре.',
 '2. В большой кастрюле смешайте слегка взбитые желтки, 3 стакана молока и сахар. Поставьте на очень маленький огонь и, взбивая, нагревайте, пока масса не загустеет. Снимите с огня д

In [8]:
sentences = titles + steps
print(len(sentences))
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in sentences]
sentences_tokenized = [tokenizer.tokenize(i) for i in sentences_tokenized]

186952


In [9]:
word_to_tag = {}
wrong_words = set()
for s in tqdm.tqdm(sentences_tokenized):
    for w in s:
        if (w not in wrong_words) and (w not in word_to_tag):
            try:
                w_tag = tag(w)
            except ValueError:
                wrong_words.add(w)
                continue
            word_to_tag[w] = w_tag

100%|██████████| 186952/186952 [00:03<00:00, 52622.06it/s]


In [10]:
sentences_tokenized_with_tags = []
for s in tqdm.tqdm(sentences_tokenized):
    cur_sentence = []
    for w in s:
        try:
            current_tag = tag(w)
        except:
            continue
        cur_sentence.append(tag(w))
    sentences_tokenized_with_tags.append(cur_sentence)

100%|██████████| 186952/186952 [05:10<00:00, 602.07it/s]


In [11]:
sentences_tokenized_with_tags[:10]

[['слоеный_ADJ', 'булка_NOUN'],
 ['итальянский_ADJ', 'персиковый_ADJ', 'мороженое_NOUN'],
 ['итальянский_ADJ', 'мороженое_NOUN', 'с_NOUN', 'лимонный_ADJ', 'цедра_NOUN'],
 ['закуска_NOUN',
  'из_ADP',
  'курица_NOUN',
  'в_NOUN',
  'лист_NOUN',
  'салат_NOUN'],
 ['мариновать_VERB', 'куриный_ADJ', 'ломтик_NOUN'],
 ['курица_NOUN',
  'с_NOUN',
  'виноград_NOUN',
  'и_SCONJ',
  'тархун_NOUN',
  'в_NOUN',
  'белый_ADJ',
  'вино_NOUN'],
 ['тарт_NOUN', 'с_NOUN', 'цитрусовый_ADJ', 'и_SCONJ', 'висок_NOUN'],
 ['овощной_ADJ', 'весенний_ADJ', 'суп_NOUN', 'с_NOUN', 'сыр_NOUN'],
 ['суп_NOUN', 'пюре_NOUN', 'гороховый_ADJ', 'с_NOUN', 'бекон_NOUN'],
 ['острый_ADJ', 'суп_NOUN', 'кукси_NOUN', 'с_NOUN', 'кимчать_VERB']]

In [28]:
model = Word2Vec(size=300, min_count=1)
model.build_vocab(sentences_tokenized_with_tags)
total_examples = model.corpus_count
model.train(sentences_tokenized_with_tags, total_examples=total_examples, epochs=300)

(702158925, 890409600)

In [38]:
model.wv.similarity(tag('огурец'), tag('помидор'))

0.50724673

In [39]:
for n in model.wv.most_similar(positive=[u'хлеб_NOUN']):
    print(n[0], n[1])

багет_NOUN 0.5961970090866089
батон_NOUN 0.5683152675628662
булка_NOUN 0.5442439317703247
чиабатт_NOUN 0.5181292295455933
тост_NOUN 0.48992249369621277
булочка_NOUN 0.4454916715621948
сэндвич_NOUN 0.38475048542022705
гренок_NOUN 0.36074358224868774
хлебный_ADJ 0.35707569122314453
хлебец_NOUN 0.35418248176574707


In [40]:
for n in model.wv.most_similar(positive=[u'лимон_NOUN']):
    print(n[0], n[1])

лайма_NOUN 0.7456248998641968
лимонный_ADJ 0.705265462398529
апельсин_NOUN 0.6387690901756287
апельсиновый_ADJ 0.558100163936615
лимонов_NOUN 0.49364572763442993
алельнин_NOUN 0.4570106863975525
лайм_NOUN 0.45244553685188293
танжело_NOUN 0.4063279628753662
апельсиновыя_NOUN 0.4026985168457031
лаймовый_ADJ 0.391347736120224


In [41]:
for n in model.wv.most_similar(positive=[u'щи_NOUN']):
    print(n[0], n[1])

суп_NOUN 0.4488198757171631
борщ_NOUN 0.44113361835479736
рассольник_NOUN 0.4297197163105011
солянка_NOUN 0.36750268936157227
уха_NOUN 0.3507572412490845
гуляш_NOUN 0.34441307187080383
калья_NOUN 0.33206242322921753
орсинь_NOUN 0.2908477187156677
тьян_NOUN 0.2791593074798584
чахохбили_NOUN 0.2714824676513672


In [42]:
for n in model.wv.most_similar(positive=[u'хек_NOUN']):
    print(n[0], n[1])

баррамундить_VERB 0.4170074462890625
амур_NOUN 0.3910447955131531
тилапия_NOUN 0.3527858257293701
палтус_NOUN 0.3493863046169281
телапия_NOUN 0.3334099352359772
гна_NOUN 0.32311418652534485
судак_NOUN 0.31817638874053955
муксун_NOUN 0.2841493487358093
форель_NOUN 0.27812644839286804
обезглавливать_VERB 0.27595198154449463


In [43]:
for n in model.wv.most_similar(positive=[u'глазурь_NOUN']):
    print(n[0], n[1])

крем_NOUN 0.5741153359413147
карамель_NOUN 0.5220332145690918
помадка_NOUN 0.5166994333267212
сироп_NOUN 0.5063947439193726
ганаш_NOUN 0.4678892195224762
торт_NOUN 0.43716004490852356
желе_NOUN 0.409631609916687
мусс_NOUN 0.38634324073791504
кекс_NOUN 0.3826807737350464
соус_NOUN 0.38038817048072815


In [44]:
for n in model.wv.most_similar(positive=[u'смузи_NOUN']):
    print(n[0], n[1])

фроузен_NOUN 0.4243203401565552
чизкек_NOUN 0.4142407774925232
сорбеты_NOUN 0.39864516258239746
десерт_NOUN 0.39526957273483276
мороженое_NOUN 0.37708765268325806
эггногий_NOUN 0.3716740310192108
грин_NOUN 0.36121702194213867
кулиса_NOUN 0.3586828410625458
мусс_NOUN 0.3572828471660614
компот_NOUN 0.35177531838417053


In [49]:
for n in model.wv.most_similar(positive=[u'помидор_NOUN']):
    print(n[0], n[1])

томат_NOUN 0.7807319164276123
цукини_NOUN 0.5298959016799927
кабачок_NOUN 0.5250635147094727
баклажан_NOUN 0.5092100501060486
огурец_NOUN 0.507246732711792
оливка_NOUN 0.49871042370796204
маслина_NOUN 0.492484450340271
болгарский_ADJ 0.45306527614593506
шампиньон_NOUN 0.44473984837532043
овощ_NOUN 0.4393238425254822


In [48]:
for n in model.wv.most_similar(positive=[u'варить_VERB']):
    print(n[0], n[1])

тушить_VERB 0.745673656463623
проваривать_VERB 0.6913928985595703
готовить_VERB 0.6619253158569336
томить_VERB 0.6392511129379272
поварить_VERB 0.6391982436180115
потушить_VERB 0.5272253751754761
кипятить_VERB 0.5116965770721436
отваривать_VERB 0.48332083225250244
подержать_VERB 0.47764837741851807
покипеть_VERB 0.47580963373184204


In [50]:
for n in model.wv.most_similar(positive=[u'сахар_NOUN']):
    print(n[0], n[1])

мед_NOUN 0.5341365337371826
сахарный_ADJ 0.4989052712917328
ванилин_NOUN 0.4900962710380554
мука_NOUN 0.48535722494125366
пудра_NOUN 0.4755537807941437
корица_NOUN 0.4504908621311188
какао_NOUN 0.4383567273616791
ваниль_NOUN 0.42336830496788025
экстракт_NOUN 0.4185176193714142
крахмал_NOUN 0.4075784683227539


In [51]:
for n in model.wv.most_similar(positive=[u'соль_NOUN']):
    print(n[0], n[1])

кайенский_ADJ 0.4898010492324829
посолить_VERB 0.4598104655742645
черный_ADJ 0.400374174118042
молотый_ADJ 0.3563736081123352
орегано_NOUN 0.3547095060348511
душистый_ADJ 0.349165678024292
ситимь_NOUN 0.34489062428474426
паприка_NOUN 0.3392484784126282
специя_NOUN 0.3340928256511688
свежемолотый_ADJ 0.3306758999824524


In [55]:
model.save('food_w2v_300.w2v')